In [ ]:
import json
import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os.path
import pytz

In [ ]:
NY_TZ = pytz.timezone('America/New_York')
time_slots = {
    'morning': datetime.time(8, 0),  # 8:00 AM
    'afternoon': datetime.time(13, 0),  # 1:00 PM
    'evening': datetime.time(18, 0),  # 6:00 PM
    'night': datetime.time(22, 0)  # 10:00 PM
}

In [ ]:
medicine_data = [
    {
        "medicine": "TAB. DEMO MEDICINE 1",
        "frequency": "1-0-1",
        "duration": "10 days",
        "special_instructions": "Before Food"
    },
    {
        "medicine": "CAP. DEMO MEDICINE 2",
        "frequency": "1-0-1",
        "duration": "10 days",
        "special_instructions": "Before Food"
    },
    {
        "medicine": "TAB. DEMO MEDICINE 3",
        "frequency": "1-1-1-1",
        "duration": "10 days",
        "special_instructions": "After Food"
    },
    {
        "medicine": "TAB. DEMO MEDICINE 4",
        "frequency": "0.5-0-0-0.5",
        "duration": "10 days",
        "special_instructions": "After Food"
    }
]

# Google Calendar API setup
SCOPES = ['https://www.googleapis.com/auth/calendar']

In [ ]:
def get_calendar_service():
    """Set up and return Google Calendar API service"""
    creds = None
    
    # Check for existing token
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_info(json.load(open('token.json')))
    
    # If credentials don't exist or are invalid
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save credentials for next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return build('calendar', 'v3', credentials=creds)

def parse_frequency(freq):
    """Parse medication frequency into morning, afternoon, evening, night doses"""
    parts = freq.split('-')
    doses = [float(part) for part in parts]
    
    # Ensure we have 4 values (morning, afternoon, evening, night)
    while len(doses) < 4:
        doses.append(0)
        
    return doses[:4]  # Return only the first 4 values

def create_medicine_reminders():
    """Create Google Calendar events for medicine reminders"""
    service = get_calendar_service()
    
    # Get today's date as the start date
    today = datetime.datetime.now(NY_TZ).date()
    
    for medicine in medicine_data:
        med_name = medicine["medicine"]
        special_instr = medicine["special_instructions"]
        
        # Parse duration (assume format is "X days")
        duration_days = int(medicine["duration"].split()[0])
        
        # Parse the frequency to determine when to take the medicine
        doses = parse_frequency(medicine["frequency"])
        
        # Time slots in order: morning, afternoon, evening, night
        slot_names = ['morning', 'afternoon', 'evening', 'night']
        
        # For each time slot with a non-zero dose
        for i, (slot_name, dose) in enumerate(zip(slot_names, doses)):
            if dose > 0:
                # Create events for each day of the duration
                for day in range(duration_days):
                    event_date = today + datetime.timedelta(days=day)
                    
                    # Get the time for this slot
                    slot_time = time_slots[slot_name]
                    
                    # Create start and end datetime
                    start_datetime = datetime.datetime.combine(event_date, slot_time).replace(tzinfo=NY_TZ)
                    end_datetime = start_datetime + datetime.timedelta(minutes=15)
                    
                    # Format dose text properly
                    if dose == 0.5:
                        dose_text = "half a tablet"
                    elif dose == 1:
                        dose_text = "1 tablet"
                    else:
                        dose_text = f"{dose} tablets"
                    
                    event = {
                        'summary': f"Take {med_name}",
                        'description': f"Take {dose_text} of {med_name}. {special_instr}",
                        'start': {
                            'dateTime': start_datetime.isoformat(),
                            'timeZone': 'America/New_York',
                        },
                        'end': {
                            'dateTime': end_datetime.isoformat(),
                            'timeZone': 'America/New_York',
                        },
                        'reminders': {
                            'useDefault': False,
                            'overrides': [
                                {'method': 'popup', 'minutes': 15},
                            ],
                        },
                    }
                    
                    # Add the event to Google Calendar
                    event = service.events().insert(calendarId='primary', body=event).execute()
                    print(f"Created event for {med_name} ({dose_text}) at {slot_name} on {event_date}")

In [ ]:
create_medicine_reminders()
print("All medicine reminders have been scheduled successfully!")